In [12]:
import os
import re
import cv2
import yaml
import shutil
import random
import torch
from tqdm import tqdm
from pathlib import Path
from ultralytics import YOLO


In [13]:
torch.cuda.set_device(0) # Set to your desired GPU number
IMG_PATH = Path(r'./data/img_dataset')
ANNOTATION_PATH = Path(r'./data/annotation')
train_val_test = (0.8,0.1,0.1)

class_dict = {category_name: idx for idx, category_name in enumerate(os.listdir(IMG_PATH))}
print(class_dict)

{'Bicycle': 0, 'Boat': 1, 'Bottle': 2, 'Bus': 3, 'Car': 4, 'Cat': 5, 'Chair': 6, 'Cup': 7, 'Dog': 8, 'Motorbike': 9, 'People': 10, 'Table': 11}


In [14]:
def copy_dataset(target_dataset):
    base_dataset = r'./data/img_dataset'
    os.makedirs(target_dataset, exist_ok=True)
    for category_name in os.listdir(base_dataset):
        os.makedirs(os.path.join(target_dataset, category_name), exist_ok=True)
        for img_name in os.listdir(os.path.join(base_dataset, category_name)):
            shutil.copy(os.path.join(base_dataset, category_name, img_name), os.path.join(target_dataset, category_name, img_name))

In [15]:
def pascal_to_yolo(image_path):
    image_name = os.path.basename(image_path)
    image_dir = os.path.dirname(image_path).replace("\\", "/")
    file_title, _ = os.path.splitext(image_name)
    yolo_anno_path = image_dir.replace("images", "labels") + f"/{file_title}.txt"

    img = cv2.imread(image_path)
    img_height, img_width = img.shape[:2]

    sub_category = image_dir.split("/")[-1]
    path_from_base = os.path.join(sub_category, image_name)
    
    original_annotation_path = os.path.join(ANNOTATION_PATH, path_from_base + ".txt")
    with open(original_annotation_path, 'r') as anno_file:
        pascal_data = anno_file.readlines()[1:]

    pascal_num = list(map(lambda s: s.strip().split(" ")[:5], pascal_data))
    
    yolo_format = []
    for indv_bounding_coordinates in pascal_num:
        indv_bounding_coordinates[0] = class_dict[indv_bounding_coordinates[0]]
        indv_bounding_coordinates[1:] = list(map(int, indv_bounding_coordinates[1:]))
        # print(indv_bounding_coordinates)

        class_category = indv_bounding_coordinates[0]
        x_ctr = min((indv_bounding_coordinates[1] + indv_bounding_coordinates[3]/2) / img_width, 1.0)
        y_ctr = min((indv_bounding_coordinates[2] + indv_bounding_coordinates[4]/2) / img_height, 1.0)
        width = min((indv_bounding_coordinates[3]) / img_width, 1.0)
        height = min((indv_bounding_coordinates[4]) / img_height, 1.0)
        yolo_format.append([class_category, x_ctr, y_ctr, width, height])
    
    os.makedirs(os.path.dirname(yolo_anno_path), exist_ok=True)
    with open(yolo_anno_path, 'w') as yolo_writer:
        for data in yolo_format:
            yolo_writer.write(f"{' '.join(list(map(str, data)))}\n")
    
    print(f"Converted to {image_path} and label {yolo_anno_path}")
    

In [ ]:
def generate_train_val_test(src_path, train_path, val_path, test_path, split_ratio, function=None, seed=None):
    if not os.path.exists(train_path):
        os.makedirs(train_path, exist_ok=True)
    if not os.path.exists(val_path):
        os.makedirs(val_path, exist_ok=True)
    if not os.path.exists(test_path):
        os.makedirs(test_path, exist_ok=True)

    if os.listdir(train_path) != []:
        shutil.rmtree(train_path)
        print(f"Removed existing train path: {train_path}")
    if os.listdir(val_path) != []:
        shutil.rmtree(val_path)
        print(f"Removed existing val path: {val_path}")
    if os.listdir(test_path) != []:
        shutil.rmtree(test_path)
        print(f"Removed existing test path: {test_path}")
    
    if seed != None:
        random.seed(seed)

    file_lists = []
    for (root, _, file_names) in os.walk(src_path):
        if file_names == []:
            continue

        sub_folder_name = os.path.basename(root)

        file_path_list = list(map(lambda path: os.path.join(sub_folder_name, path), file_names))

        random.shuffle(file_path_list)
        file_lists.append(file_path_list)

    for category_specific_list in file_lists:
        for file_name in category_specific_list:
            random_num = random.random()

            if random_num < split_ratio[0]:
                move_path = os.path.join(train_path, os.path.dirname(file_name))
            elif random_num < sum(split_ratio[:2]):
                move_path = os.path.join(val_path, os.path.dirname(file_name)) 
            else:
                move_path = os.path.join(test_path, os.path.dirname(file_name))

            os.makedirs(move_path, exist_ok=True)            
            original_path = os.path.join(src_path, file_name)
            new_name = os.path.join(move_path, os.path.basename(file_name))
            shutil.copy(original_path, move_path)
            if function != None:
                function(move_path)
            else:
                shutil.copy(original_path, move_path)

            pascal_to_yolo(new_name)
            
            print(original_path)
            print(new_name)


In [17]:
# check for different file names for images and labels

def check_file_names(folder): # test_data or train_val
    img_folder = os.path.join(folder, "images")
    label_folder = os.path.join(folder, "labels")
    train_img = os.path.join(img_folder, "train")
    val_img = os.path.join(img_folder, "val")
    train_label = os.path.join(label_folder, "train")
    val_label = os.path.join(label_folder, "val")

    for (root, dirname, filename) in os.walk(train_img):
        for objtype in dirname:
            obj_path = os.path.join(root, objtype)
            for (root, dirname, file_names) in os.walk(obj_path):
                for file_name in file_names:
                    file_name = os.path.splitext(file_name)[0]
                    label_name = os.path.join(train_label, objtype, file_name + ".txt")
                    if not os.path.exists(label_name):
                        print(f"Missing label file for {file_name} in {label_name}")
    
    for (root, dirname, filename) in os.walk(val_img):
        for objtype in dirname:
            obj_path = os.path.join(root, objtype)
            for (root, dirname, file_names) in os.walk(obj_path):
                for file_name in file_names:
                    file_name = os.path.splitext(file_name)[0]
                    label_name = os.path.join(val_label, objtype, file_name + ".txt")
                    if not os.path.exists(label_name):
                        print(f"Missing label file for {file_name} in {label_name}")


In [18]:
# check label files if width and height >1
def check_label_files(folder):

    label_folder = os.path.join(folder, "labels")
    print(label_folder)
    for root, dirname, files in os.walk(label_folder):
        if files == []:
            continue
        for file in files:
            if "cache" in file:
                continue
            
            label_path = os.path.join(root, file)
            with open(label_path, 'r') as f:
                lines = f.readlines()
                new_lines = []
                for line in lines:
                    values = list(map(float, line.strip().split()))
                    if values[1] > 1 or values[2] > 1 or values[3] > 1 or values[4] > 1:
                        values[1] = 1.0 if values[1] > 1 else values[1]
                        values[2] = 1.0 if values[2] > 1 else values[2]
                        values[3] = 1.0 if values[3] > 1 else values[3]
                        values[4] = 1.0 if values[4] > 1 else values[4]
                        print(f"Invalid bounding box in {label_path}: {line.strip()}")
                    if values[1] < 0 or values[2] < 0 or values[3] < 0 or values[4] < 0:
                        values[1] = 0.0 if values[1] < 0 else values[1]
                        values[2] = 0.0 if values[2] < 0 else values[2]
                        values[3] = 0.0 if values[3] < 0 else values[3]
                        values[4] = 0.0 if values[4] < 0 else values[4]
                        print(f"Invalid bounding box in {label_path}: {line.strip()}")
                    if len(values) != 5:
                        print(f"Invalid label format in {label_path}: {line.strip()}")
                    new_line = " ".join(str(v) for v in values) + "\n"
                    new_lines.append(new_line)
                # Overwrite the original file with the corrected content
                with open(label_path, 'w') as f:
                    f.writelines(new_lines)
                


In [19]:
from preprocessors.retinex.msr import process_img

WEIGHTS = {
    30: (0.7, 0.7),
    70: (0.5, 0,3),
    110: (0.3, 0.2)
}

In [ ]:
custom_dataset_path = './data/msrcc_train_val/images'

generate_train_val_test(
    src_path='./data/img_dataset',
    train_path=os.path.join(custom_dataset_path, 'train'),
    val_path=os.path.join(custom_dataset_path, 'val'),
    test_path='./data/msrcc_test_data/images',
    split_ratio=train_val_test,
    function=process_img
)

Removed existing train path: ./data/msrcc_train_val/images\train
Removed existing val path: ./data/msrcc_train_val/images\val
Removed existing test path: ./data/msrcc_test_data/images
Converted to ./data/msrcc_train_val/images\val\Bicycle\2015_00246.jpg and label ./data/msrcc_train_val/labels/val/Bicycle/2015_00246.txt
./data/img_dataset\Bicycle\2015_00246.jpg
./data/msrcc_train_val/images\val\Bicycle\2015_00246.jpg
Converted to ./data/msrcc_train_val/images\train\Bicycle\2015_00627.jpg and label ./data/msrcc_train_val/labels/train/Bicycle/2015_00627.txt
./data/img_dataset\Bicycle\2015_00627.jpg
./data/msrcc_train_val/images\train\Bicycle\2015_00627.jpg
Converted to ./data/msrcc_train_val/images\train\Bicycle\2015_00073.jpg and label ./data/msrcc_train_val/labels/train/Bicycle/2015_00073.txt
./data/img_dataset\Bicycle\2015_00073.jpg
./data/msrcc_train_val/images\train\Bicycle\2015_00073.jpg
Converted to ./data/msrcc_train_val/images\train\Bicycle\2015_00282.jpg and label ./data/msrcc_t

In [21]:
for (root, dirs, file_names) in os.walk(custom_dataset_path):
    for file in file_names:
        file_path = os.path.join(root, file)
        if "cache" in file:
            os.remove(file_path)
            continue

        processed_img = process_img(file_path, input_weights=WEIGHTS)
        cv2.imwrite(file_path, processed_img)

In [22]:
model = YOLO('yolov8n.yaml')
model.load('/snapshots/yolov8n.pt')
model.to('cuda')

print(model.info())

Transferred 355/355 items from pretrained weights
YOLOv8n summary: 129 layers, 3,157,200 parameters, 3,157,184 gradients, 8.9 GFLOPs
(129, 3157200, 3157184, 8.8575488)


In [23]:
train_config = {
    'data': 'data.yaml',
    'epochs': 100,
    'patience': 10,
    'batch': 16,
    'imgsz': 640,
    'save': True,
    'save_period': 1,
    'project': r'./model_runs',
    'name': 'default_augmentation_train',
    'exist_ok': False,
    'seed': 0,
    'resume': True,
    'lr0': 0.1, 
    'lrf': 0.001,
    'dropout': 0.0,
    'val': True,
    'plots':True,
}

val_config = {
    'data': 'data.yaml',
    'batch': 16,
    'imgsz': 640,
    'conf': 0.25,
    'iou': 0.5,
    'max_det': 100,
    'plots': True,
    'split': 'val',
    'project': r'./model_runs',
    'name': 'default_augmentation_val'
}

In [24]:
model.train(
    **train_config
)


New https://pypi.org/project/ultralytics/8.3.111 available  Update with 'pip install -U ultralytics'
engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=data.yaml, epochs=100, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=1, cache=False, device=cuda:0, workers=8, project=./model_runs, name=default_augmentation_train, exist_ok=False, pretrained=/snapshots/yolov8n.pt, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=None, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=T

100%|██████████| 5.35M/5.35M [00:00<00:00, 9.88MB/s]


AMP: checks passed 


train: Scanning C:\Users\DuenoHfao\Desktop\Work\SCVU\mlgp_full\data\train_val\labels\train\Bicycle... 5935 images, 1 backgrounds, 0 corrupt: 100%|██████████| 5935/5935 [00:07<00:00, 769.89it/s] 

train: WARNING  C:\Users\DuenoHfao\Desktop\Work\SCVU\mlgp_full\data\train_val\images\train\Bottle\2015_01610.jpg: corrupt JPEG restored and saved
train: WARNING  C:\Users\DuenoHfao\Desktop\Work\SCVU\mlgp_full\data\train_val\images\train\Bottle\2015_01716.jpg: corrupt JPEG restored and saved
train: WARNING  C:\Users\DuenoHfao\Desktop\Work\SCVU\mlgp_full\data\train_val\images\train\Car\2015_02634.jpg: corrupt JPEG restored and saved
train: WARNING  C:\Users\DuenoHfao\Desktop\Work\SCVU\mlgp_full\data\train_val\images\train\Car\2015_02636.jpg: corrupt JPEG restored and saved
train: WARNING  C:\Users\DuenoHfao\Desktop\Work\SCVU\mlgp_full\data\train_val\images\train\Car\2015_02637.jpg: corrupt JPEG restored and saved
train: WARNING  C:\Users\DuenoHfao\Desktop\Work\SCVU\mlgp_full\data\train_val\images\train\Car\2015_02639.jpg: corrupt JPEG restored and saved
train: WARNING  C:\Users\DuenoHfao\Desktop\Work\SCVU\mlgp_full\data\train_val\images\train\Car\2015_02641.jpg: corrupt JPEG restored and

...

                   all        738       2507      0.741      0.622      0.704      0.447
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 35, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



45 epochs completed in 0.510 hours.
Optimizer stripped from model_runs\default_augmentation_train\weights\last.pt, 6.2MB
Optimizer stripped from model_runs\default_augmentation_train\weights\best.pt, 6.2MB

Validating model_runs\default_augmentation_train\weights\best.pt...
Ultralytics 8.3.107  Python-3.10.5 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8192MiB)
YOLOv8n summary (fused): 72 layers, 3,007,988 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:08<00:00,  2.99it/s]


                   all        738       2507      0.765      0.627      0.716      0.453
               Bicycle         83        126      0.759      0.683       0.76      0.498
                  Boat         69        156      0.786      0.634      0.765      0.411
                Bottle         76        194      0.698       0.56      0.629      0.404
                   Bus         42         51      0.841      0.831      0.927      0.707
                   Car        141        336      0.803       0.72      0.801       0.52
                   Cat         80        100      0.745       0.61      0.687      0.387
                 Chair        139        255      0.716      0.484      0.586      0.358
                   Cup         81        161      0.876      0.553      0.688      0.444
                   Dog         72         79      0.678      0.693      0.731      0.522
             Motorbike         81        141      0.822      0.631      0.764      0.482
                Peopl

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001CE9626EFE0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.04504

In [ ]:
model.val(
    **val_config
)